# PROJET 6 - DÉTECTER LES FAUX BILLETS

Votre société de consulting informatique vous propose une nouvelle mission au ministère de l'Intérieur, dans le cadre de la lutte contre la criminalité organisée, à l'Office central pour la répression du faux monnayage. Votre mission si vous l'acceptez : **créer un algorithme de détection de faux billets**.  
Vous vous voyez déjà en grand justicier combattant sans relâche la criminalité organisée en pianotant à mains de maître votre ordinateur, pour façonner ce fabuleux algorithme  qui traquera la moindre fraude et permettra de mettre à jour les réseaux secrets de faux-monnayeurs ! La classe, non ?  
Bon, si on retombait les pieds sur terre ? Travailler pour la police judiciaire, c'est bien, mais vous allez devoir faire appel à vos connaissances en statistiques, alors on y va !

## Importation des modules

In [1]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.mlab as mlab
import seaborn as sns
import numpy as np
import pandas as pd

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn import decomposition, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans 
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score

## I. Chargement des données

In [2]:
data = pd.read_csv('/Users/anissa/P6/notes.csv')
data.head()

,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
0,True,171.81,104.86,104.95,4.52,2.89,112.83
1,True,171.67,103.74,103.70,4.01,2.87,113.29
2,True,171.83,103.76,103.76,4.40,2.88,113.84
3,True,171.80,103.78,103.65,3.73,3.12,113.63
4,True,172.05,103.70,103.75,5.04,2.27,113.55


La _longueur du billet_ (en mm) : **length**  
La _hauteur du billet_ (mesurée sur le côté gauche, en mm) : **height_left**  
La _hauteur du billet_ (mesurée sur le côté droit, en mm) : **height_right**  
La _marge entre le bord supérieur_ du billet et l'image de celui-ci (en mm) : **margin_up**  
La _marge entre le bord inférieur_ du billet et l'image de celui-ci (en mm) : **margin_low**  
La _diagonale du billet_ (en mm) : **diagonal**  

In [3]:
data.shape

(170, 7)

Le df est composé de **170 billets** et de **7 variables** : 
* 6 variables quantititaves : _diagonal, height_left, height_right, margin_low, margin_up et length_
* 1 qualitative : *is_genuine*

In [4]:
# Recherche des valeurs manquantes
print('Valeurs manquantes :\n' + str(data.isnull().sum()))

Valeurs manquantes :
is_genuine      0
diagonal        0
height_left     0
height_right    0
margin_low      0
margin_up       0
length          0
dtype: int64


In [5]:
# Recherche des valeurs dupliquées
print('Valeurs dupliquées : ', data.duplicated().sum())

Valeurs dupliquées :  0


## II. Analyses univariées et bivariées

In [6]:
description = data.groupby('is_genuine').describe().T
print(description)

is_genuine               False       True 
diagonal     count   70.000000  100.000000
             mean   171.889857  171.976100
             std      0.297426    0.307981
             min    171.380000  171.040000
             25%    171.682500  171.790000
             50%    171.875000  172.005000
             75%    172.047500  172.162500
             max    173.010000  172.750000
height_left  count   70.000000  100.000000
             mean   104.230429  103.951500
             std      0.213130    0.296251
             min    103.780000  103.230000
             25%    104.082500  103.740000
             50%    104.215000  103.915000
             75%    104.377500  104.145000
             max    104.720000  104.860000
height_right count   70.000000  100.000000
             mean   104.145571  103.775900
             std      0.253152    0.292406
             min    103.440000  103.140000
             25%    103.982500  103.557500
             50%    104.170000  103.760000
           

Notre échantillon contient 170 billets : 100 vrais billets et 70 faux.

In [ ]:
# Matrice de corrélation
sns.set_theme(style="white")
d = data
corr = d.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))


f, ax = plt.subplots(figsize=(25, 15))


cmap = sns.diverging_palette(230, 20, as_cmap=True)


sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)
plt.title("Matrice de corrélation", size=15)
plt.savefig('/Users/anissa/P6_matrice_correlation.jpg', dpi=1200)
plt.show()

Les variables "length" et "margin_low" sont les variables **les plus corrélées à la variable "is_genuine"**. Ce qui signifie que ce sont ces 2 variables qui sont les plus **importantes** lors de la détection de faux billets.  
On constate une **très faible corrélation** entre "margin_up" et "diagonal".  
Une **forte corrélation positive** est constatée entre "height_right" et "height_left" : les deux variables varient dans le même sens.   
Une **corrélation négative** existe entre "margin_low" et "length" : lorsqu'une des variables augmente, l'autre diminue.

On constate bien une différence significative entre les données des vrais et des faux billets.  
La variable "length" montre que la taille des faux billets est inférieure à celle des vrais.

In [ ]:
# Comparaison entre les vrais et faux billets (violinplot)
sns.set_palette('Spectral')
medianprops = dict(linewidth=2, color='grey')
meanprops={"marker":"o",
                       "markerfacecolor":"yellow", 
                       "markeredgecolor":"yellow",
                      "markersize":"5"}

for variable in data.columns[1:7]:
    sns_plot=sns.violinplot(
        x = data[variable], 
        y = data.is_genuine,
        orient = "h",
        medianprops = medianprops, showmeans=True, meanprops=meanprops,
        width = .5)
    plt.show()

In [ ]:
# Comparaison entre les dimensions des vrais et faux billets (boxplot)
medianprops = dict(linewidth=2, color='grey')
meanprops={"marker":"o",
                       "markerfacecolor":"yellow", 
                       "markeredgecolor":"yellow",
                      "markersize":"5"}

for variable in data.columns[1:7]:
    sns_plot=sns.boxplot(
        x = data[variable], 
        y = data.is_genuine,
        orient = "h",
        medianprops = medianprops, showmeans=True, meanprops=meanprops,
        width = .5)
    plt.show()

On constate ici, comme pour la matrice de corrélation, que les valeurs les plus discriminantes sont **"length"** et **"margin_low"** : différences importantes entre les dimensions et données plus hétérogènes.

## III. Analyse en Composantes Principales

_Source :_ http://eric.univ-lyon2.fr/~ricco/tanagra/fichiers/fr_Tanagra_ACP_Python.pdf

In [ ]:
# Paramétrage de l'environnement

def display_circles(pcs, n_comp, pca, axis_ranks, labels=None, label_rotation=0, lims=None):
    for d1, d2 in axis_ranks: # On affiche les 3 premiers plans factoriels, donc les 6 premières composantes
        if d2 < n_comp:

            # initialisation de la figure
            fig, ax = plt.subplots(figsize=(10,10))

            # détermination des limites du graphique
            if lims is not None :
                xmin, xmax, ymin, ymax = lims
            elif pcs.shape[1] < 30 :
                xmin, xmax, ymin, ymax = -1, 1, -1, 1
            else :
                xmin, xmax, ymin, ymax = min(pcs[d1,:]), max(pcs[d1,:]), min(pcs[d2,:]), max(pcs[d2,:])

            # affichage des flèches
            # s'il y a plus de 30 flèches, on n'affiche pas le triangle à leur extrémité
            if pcs.shape[1] < 30 :
                plt.quiver(np.zeros(pcs.shape[1]), np.zeros(pcs.shape[1]),
                   pcs[d1,:], pcs[d2,:], 
                   angles='xy', scale_units='xy', scale=1, color="grey")
                # (voir la doc : https://matplotlib.org/api/_as_gen/matplotlib.pyplot.quiver.html)
            else:
                lines = [[[0,0],[x,y]] for x,y in pcs[[d1,d2]].T]
                ax.add_collection(LineCollection(lines, axes=ax, alpha=.1, color='black'))
            
            # affichage des noms des variables  
            if labels is not None:  
                for i,(x, y) in enumerate(pcs[[d1,d2]].T):
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax :
                      plt.text(x, y, labels[i], fontsize='14', ha='center', va='center', rotation=label_rotation, color="blue", alpha=0.5)
            
            # affichage du cercle
            an = np.linspace(0, 2 * np.pi, 100)  # Add a unit circle for scale
            plt.plot(np.cos(an), np.sin(an))
            plt.axis('equal')

            # définition des limites du graphique
            plt.xlim(xmin, xmax)
            plt.ylim(ymin, ymax)
        
            # affichage des lignes horizontales et verticales
            plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Cercle des corrélations (F{} et F{})".format(d1+1, d2+1))
            plt.show(block=False)
        
def display_factorial_planes(X_projected, n_comp, pca, axis_ranks, labels=None, alpha=1, illustrative_var=None):
    for d1,d2 in axis_ranks:
        if d2 < n_comp:
 
            # initialisation de la figure       
            fig = plt.figure(figsize=(10,10))
        
            # affichage des points
            if illustrative_var is None:
                plt.scatter(X_projected[:, d1], X_projected[:, d2], alpha=alpha)
            else:
                illustrative_var = np.array(illustrative_var)
                for value in np.unique(illustrative_var):
                    selected = np.where(illustrative_var == value)
                    plt.scatter(X_projected[selected, d1], X_projected[selected, d2], alpha=alpha, label=value)
                plt.legend()

            # affichage des labels des points
            if labels is not None:
                for i,(x,y) in enumerate(X_projected[:,[d1,d2]]):
                    plt.text(x, y, labels[i],
                              fontsize='14', ha='center',va='center') 
                
            # détermination des limites du graphique
            boundary = np.max(np.abs(X_projected[:, [d1,d2]])) * 1.1
            plt.xlim([-boundary,boundary])
            plt.ylim([-boundary,boundary])
        
            # affichage des lignes horizontales et verticales
            plt.plot([-100, 100], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-100, 100], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Projection des individus (sur F{} et F{})".format(d1+1, d2+1))
            plt.show(block=False)

In [ ]:
data_acp = data.drop(columns='is_genuine') # on supprime la variable qualitative pour l'ACP

print(data_acp.shape) # dimension de la matrice
n = data_acp.shape[0] # nombre d'observations
p = data_acp.shape[1] # nombre de variables

In [ ]:
sc = StandardScaler()
Z = sc.fit_transform(data_acp)

print(Z)

In [ ]:
acp = PCA() # instanciation
print(acp)

In [ ]:
coord = acp.fit_transform(Z)
print(acp.n_components_) # nombre de composantes calculées

In [ ]:
# Variance expliquée
print(acp.explained_variance_)

In [ ]:
# Valeur corrigée
eigval = (n-1)/n*acp.explained_variance_
print(eigval)

In [ ]:
# Proportion de valeurs expliquées
ratio = acp.explained_variance_ratio_ * 100
print(acp.explained_variance_ratio_)

In [ ]:
# Eboulis des valeurs propres
fig, ax = plt.subplots(figsize=(10,10))
scree = acp.explained_variance_ratio_*100

plt.bar(np.arange(len(scree))+1, scree)
plt.plot(np.arange(len(scree))+1, scree.cumsum(),marker='o', color='r')
plt.xlabel("Rang de l'axe d'inertie")
plt.ylabel("Pourcentage d'inertie")

plt.title("Eboulis des valeurs propres")

plt.savefig('/Users/anissa/P6_eboulis.jpg', dpi=1200)
plt.show(block=False)

Le critère du Kaiser nous conduit à retenir les deux premiers axes. En effet le premier axe retient 47.4% de l’inertie totale quant à l’axe 2 retient tout de même 22% de l’inertie, ce qui n’est pas négligeable. Et qui conduit à un taux d’inertie expliquée de 69,4%, ce qui est un très bon résultat.

In [ ]:
# Contribution des individus dans l'inertie totale
di = np.sum(Z**2,axis=1)
print(pd.DataFrame({'ID':data_acp.index,'d_i':di}))

In [ ]:
# Qualité de représentation des individus - COS2
cos2 = coord**2

for j in range(p):
  cos2[:,j] = cos2[:,j]/di

qualite = pd.DataFrame({'id':data_acp.index,'COS2_F1':cos2[:,0],'COS2_F2':cos2[:,1]})

print(qualite)

In [ ]:
print(qualite.sort_values('COS2_F1', ascending=False).head(10))

In [ ]:
print(qualite.sort_values('COS2_F2', ascending=False).head(10))

In [ ]:
#vérifions la théorie - somme en ligne des cos2 = 1
print(np.sum(cos2,axis=1))

In [ ]:
# Contribution aux axes
ctr = coord**2
for j in range(p):
  ctr[:,j] = ctr[:,j]/(n*eigval[j]) # eigval = c. 40 (variance expliquée et corrigé)

contribution = pd.DataFrame({'id':data_acp.index,'CTR_F1':ctr[:,0],'CTR_F2':ctr[:,1]})
print(contribution)

In [ ]:
print(contribution.sort_values('CTR_F1', ascending=False).head(10))

In [ ]:
print(contribution.sort_values('CTR_F2', ascending=False).head(10))

In [ ]:
# Vérifions la théorie
print(np.sum(ctr,axis=0))

In [ ]:
# Le champs components_ de l'objet ACP
print(acp.components_)

In [ ]:
# Racine carrée des valeurs propres
sqrt_eigval = np.sqrt(eigval)

# Corrélation des variables avec les axes
corvar = np.zeros((p,p))
for k in range(p):
  corvar[:,k] = acp.components_[k,:] * sqrt_eigval[k]

# Afficher la matrice des corrélations variables x facteurs    
print(corvar)

In [ ]:
# On affiche pour les deux premiers axes
print(pd.DataFrame({'id':data_acp.columns,'COR_1':corvar[:,0],'COR_2':corvar[:,1]}))

In [ ]:
data_acp.columns

In [ ]:
# Cercle des corrélations
pcs = acp.components_
features = data_acp.columns
display_circles(pcs, p, acp, [(0,1)], labels = np.array(features))

Sur F1, nous voyons les variables liées aux hauteurs et aux marges.  
La variable "length" est corrélée à F2.

In [ ]:
# Projection des individus
display_factorial_planes(coord, p, acp, [(0,1)], illustrative_var=data['is_genuine'])

Contient 69,4% de l’information : on observe très distinctement les deux groupes

In [ ]:
# Cosinus carré des variables
cos2var = corvar**2
print(pd.DataFrame({'id':data_acp.columns,'COS2_1':cos2var[:,0],'COS2_2':cos2var[:,1]}))

In [ ]:
# Contributions
ctrvar = cos2var
for k in range(p):
  ctrvar[:,k] = ctrvar[:,k]/eigval[k]

# On n'affiche que pour les deux premiers axes
print(pd.DataFrame({'id':data_acp.columns,'CTR_1':ctrvar[:,0],'CTR_2':ctrvar[:,1]}))

### IV. Classification : k-means

Nous allons séparer notre échantillon en 2 groupes : variances égales et intertie minime.

In [ ]:
X = data.values

km = KMeans(n_clusters=2)
km.fit(X)
clustersk = km.labels_

data_clusterk = pd.DataFrame({'clusters_kmeans' : clustersk})
data = data.join(data_clusterk)
illustrative_var = data['clusters_kmeans']

In [ ]:
description = data.groupby('clusters_kmeans').describe().transpose()
print(description)

In [ ]:
description2 = data.groupby('is_genuine').describe().transpose()
print(description2)

In [ ]:
# Moyennes des dimensions pour les 2 clusters
data.groupby('clusters_kmeans').mean().drop("is_genuine",axis=1)

In [ ]:
# Moyennes des dimensions pour les 2 clusters (k-means)
data.groupby('is_genuine').mean().drop("clusters_kmeans",axis=1)

Cette comparaison permet de voir que le clustering a bien fonctionné : la fonction a crée deux groupes qui sont très proches des groupes vrais et faux. On réalise maintenant une matrice de confusion afin de vérifier la qualité de notre classification.

In [ ]:
# Matrice de confusion
df_confusion = pd.crosstab(data.is_genuine,data.clusters_kmeans)
df_confusion

In [ ]:
# Représentation graphique de la matrice de confusion : heat map
sns.heatmap(df_confusion, annot=True)

plt.title("Matrice de confusion", size=15)
plt.savefig('/Users/anissa/P6_matrice_confusion.jpg', dpi=1200)
plt.show()

69 vrais négatifs et 100 vrais positifs ainsi qu'1 faux négatif (négatif à tord).  
Pour comparaison, en regardant la variable "is_genuine" de notre échantillon, il y a 100 vrais billets et 70 faux billets.

In [ ]:
# Projection des individus
display_factorial_planes(coord, p, acp, [(0,1)],
                         illustrative_var=data.clusters_kmeans)

### V. Modélisation

_sources :_ 
* https://www.youtube.com/watch?v=xYDgnjtVFgU
* http://eric.univ-lyon2.fr/~ricco/tanagra/fichiers/fr_Tanagra_Python_Regression_Logistique.pdf
* https://towardsdatascience.com/evaluating-machine-learning-classification-problems-in-python-5-1-metrics-that-matter-792c6faddf5

In [ ]:
# Variables explicatives
X = data[['length', 'height_left', 'height_right', 'margin_low', 'margin_up', 'diagonal']]

# Variable à expliquer
y = data.is_genuine

In [ ]:
# Partition aléatoire du jeu de données en 80% pour créer le modèle, 20% pour tester le modèle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
# Régression logistique
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# Stockage de prédictions
predictions = model.predict(X_test)
print(predictions)
print(y_test)

In [ ]:
# Evaluation du modèle
print(classification_report(y_test, predictions))
print(accuracy_score(y_test,predictions))

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, predictions)

# Assignation du nom des colonnes
cm_df = pd.DataFrame(cm, 
            columns = ['Predicted Negative', 'Predicted Positive'],
            index = ['Actual Negative', 'Actual Positive'])
cm_df

## VI. Calculs manuels des indicateurs

In [ ]:
# Vrai positif (true positive)
TP = cm_df.iloc[1,1]

# Vrai négatif (true negative)
TN = cm_df.iloc[0,0]

# Faux négatif (false negative)
FN = cm_df.iloc[0,1]

# Faux positif (false positive)
FP = cm_df.iloc[1,0]

In [ ]:
# Calcul de la sensibilité (taux de vrais positifs)
conf_sensitivity = (TP / float(TP + FN))

# Calcul de la spécificité (taux de vrais négatifs)
conf_specificity = (TN / float(TN + FP))

print("sensivity =",conf_sensitivity)
print("specificity =",conf_specificity)

In [ ]:
# Calcul de la précision (accuracy score)
conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
conf_accuracy

In [ ]:
# Moyenne entre la sensibilité et la spécificité (balanced accuracy)
balanced_accuracy_score(y_test, predictions, sample_weight=None, adjusted=False)

### VII. Test nouvel échantillon

In [ ]:
new_sample = pd.read_csv('/Users/anissa/P6/example.csv')
new_sample.head()

In [ ]:
# Préparation des données
new_predict = new_sample[['length', 'height_left', 'height_right', 'margin_low', 'margin_up', 'diagonal']]

# Application du modèle
predict = model.predict(new_predict)

# Probabilités
model.predict_proba(new_predict)

In [ ]:
# Ordre de lecture des probabilités
model.classes_

In [ ]:
# Calcul des probas d'affectaion sur l'ech. à prédire
probas_ex = model.predict_proba(new_predict)

new_predict['Probas_faux'] = probas_ex[:,0]
new_predict['Probas_vrais'] = probas_ex[:,1]
new_predict

In [ ]:
# Ajout du resultat et création du df
prediction=pd.DataFrame({'id': new_sample.id,
                        'probalité_true' : new_predict.Probas_vrais,
                        'probalité_false' : new_predict.Probas_faux,
                        'prédiction' : predict})

prediction

Notre échantillon contient donc 3 faux billets et 2 vrais.